In [ ]:
from transformers import AutoModel, AutoModelForSequenceClassification
import torch

In [ ]:
# Исходная TinyBERT (только backbone)
original_model_name = "google/bert_uncased_L-2_H-128_A-2"
original_model = AutoModel.from_pretrained(original_model_name)

# Fine-tuned TinyBERT (с classifier)
finetuned_model_name = "philschmid/tiny-bert-sst2-distilled"
finetuned_model = AutoModelForSequenceClassification.from_pretrained(finetuned_model_name)

# Разделяем части модели
parts_original = {
    "encoder": dict(original_model.encoder.named_parameters()),
    "pooler": dict(original_model.pooler.named_parameters())
}

parts_finetuned = {
    "encoder": dict(finetuned_model.bert.encoder.named_parameters()),
    "pooler": dict(finetuned_model.bert.pooler.named_parameters()),
    "classifier": dict(finetuned_model.classifier.named_parameters())
}

def analyze_parts(parts1, parts2, tol=1e-6):
    """
    Сравнивает части моделей.
    Возвращает словарь с результатами:
        - 'exists_in_other': есть ли слой в другой модели
        - 'shape_match': совпадает ли размерность
        - 'changed': отличаются ли значения
    """
    analysis = {}

    for part_name, layers1 in parts1.items():
        analysis[part_name] = {}
        layers2 = parts2.get(part_name, {})
        for layer_name, param1 in layers1.items():
            info = {}
            if layer_name in layers2:
                param2 = layers2[layer_name]
                info['exists_in_other'] = True
                info['shape_match'] = param1.shape == param2.shape
                info['changed'] = not torch.allclose(param1, param2, atol=tol)
            else:
                info['exists_in_other'] = False
                info['shape_match'] = None
                info['changed'] = None
            analysis[part_name][layer_name] = info
    return analysis

analysis_result = analyze_parts(parts_finetuned, parts_original)

# Вывод
for part, layers in analysis_result.items():
    print(f"\n=== {part.upper()} ===")
    for layer, info in layers.items():
        print(f"{layer}: exists_in_other={info['exists_in_other']}, "
              f"shape_match={info['shape_match']}, changed={info['changed']}")
